# Dynamics: simulation and control
This notebook focuses on the simulation of polyarticulated systems, with final exercices where simple control laws have to be designed. The first part deals with collision detection using the hppfcl module of pinocchio. We then build a complete simulation engine for rigid unilateral contacts. 

In [1]:
import gepetuto.magic

## Set up

We will use several models in this tutorial:
- a simple scene with 3 convex objects (buildSceneThreeBodies)
- a variation of this first scene with more objects and walls (buildScenePillsBox)
- a stack of cubes of various size (buildSceneCubes)
- a robot hand

In [2]:
from tp4.scenes import buildSceneThreeBodies, buildScenePillsBox, buildSceneCubes

We rely on the HPP-FCL module of Pinocchio, which compute collision between geometries.

In [3]:
import hppfcl
import pinocchio as pin
import numpy as np
import time
from supaero2024.meshcat_viewer_wrapper import MeshcatVisualizer
from tp4 import compatibility

## A bas collisions

The goad of this section is to introduce a simple example of collision distances between bodies, and the underlying notions of witness points and segment and normal direction.

Let's build a simple scene with 3 objects and display their proximity.

In [4]:
# %load tp4/generated/example_display_witness_build
# Build a scene
model,geom_model = buildSceneThreeBodies()
data = model.createData()
geom_data = geom_model.createData()

# Start meshcat
viz = MeshcatVisualizer(model=model, collision_model=geom_model,
                        visual_model=geom_model)


In [5]:
q=pin.randomConfiguration(model)
viz.display(q)

In [6]:
viz.viewer.jupyter_cell()

We can compute the distances between the 3 objects. Let's do it and show the closest points.

In [7]:
from tp4.display_witness import DisplayCollisionWitnessesInMeshcat

In [8]:
# %load tp4/generated/example_display_witness_witness
# Build the viewer add-on to display the witnesses.
mcWitnesses = DisplayCollisionWitnessesInMeshcat(viz)


In [9]:
pin.computeDistances(model,data,geom_model,geom_data,q)
mcWitnesses.displayDistances(geom_data)

Each collision pair corresponds to a pair of closets points, respectively located at the surface of the collision geometries if they are not colliding. These points are sometime called the *witness* points.

The witness points are connected by the *witness* segment. This segment is normal to the two collision surfaces in the case the surface is smooth around the witness point. The normalized direction is called the collision *normal*. Its orientation is a convention (the most logical convention is to go from body 1 to body 2 of the collision pair).

Let's move the objects to better see these witness elements.

In [10]:
# %load tp4/generated/example_display_witness_trajectory
v = (np.random.rand(model.nv)*2-1)*1e-3
r0 = [ np.linalg.norm(q[7*i:7*i+3]) for i in range(model.nq//7) ]
for t in range(100):

    # Update the robot position along an arbitrary trajectory
    q = pin.integrate(model,q,v*10)
    for i in range(model.nq//7):
        q[7*i:7*i+3] *= r0[i]/np.linalg.norm(q[7*i:7*i+3])
    viz.display(q)

    # Display the witness points
    pin.computeDistances(model,data,geom_model,geom_data,q)
    mcWitnesses.displayDistances(geom_data)

    time.sleep(.01)


## Details about Pinocchio HPP-FCL 
Let's see now how to extract the contact and distance information from Pinocchio and the module HPP-FCL, and how to store this in a proper data structure.

### Geometry model
The geometry model contains a set of object, each described by a name, a geometry primitive and a placement with respect to a parent joint.

In [11]:
geom1 = geom_model.geometryObjects[0]
geom1.name, geom1.parentJoint, geom1.geometry, geom1.placement

In addition, we also store the pairs of geometry objects that should be considered when evaluating collisions and distances.

In [12]:
list(geom_model.collisionPairs)

### HPP-FCL computeDistances and computeCollisions
The geometry algorithms are implemented in HPP-FCL under two different sets of functions, which respectively compute the distance between bodies, and the collision between bodies.
When computing the distance, a unique pair of witness points is produced, and the signed distance is also evaluated.
When computing the collision, an effort is made to compute all the contact points, and early stop can be activated to reduce the algorithm cost as soon as a collision is found. 
All in all, for this initiation, both can be considered quite similarly.

Both functions are parametrized by a *request* object, and write their output in a *result* object. *Request* and *result* objects are preallocated in the geometry data, one of each for each pair of collisions. If you activate or deactivate a collision pair, you have to regenerate these objects (and so if you add a new geometry object in the list). 

In [13]:
len(geom_model.collisionPairs), len(geom_data.collisionRequests), len(geom_data.distanceResults)

The placement of the geometry objects with respect to world frame are stored in geom_data.oMg, and computed as
$$^oM_g(q) = ^oM_i(q) ^iM_g$$
with $^oM_g$ the placement of the object wrt world, $^oM_i$ the placement of the parent joint and $^iM_g$ the (fixed) placement of the object wrt the parent joint.

In [14]:
len(geom_model.geometryObjects),len(geom_data.oMg)

This computation is triggered by *pin.updateGeometryPlacements*, after forward kinematics as been run or by forcing the refresh of the forward kinematics.

In [15]:
pin.updateGeometryPlacements(model,data,geom_model,geom_data,q)

The computation of the distances and collisions is triggered by their respective function and by default forces the kinematic update..

In [16]:
pin.computeCollisions(model,data,geom_model,geom_data,q)
pin.computeDistances(model,data,geom_model,geom_data,q)

These two functions actually iterate over each pair and run *pin.computeDistance* or *pin.computeCollision*, which in turn runs *hppfcl.distance* and *hppfcl.collide*. 
HPP-FCL only works with geometry placements, ignoring the kinematics and the configuration space which is provided by Pinocchio.
All these functions have very similar signatures, and Pinocchio is mostly doing a basing wrapping around the HPP-FCL library and gently connected the forward kinematics with the collision algorithms. 

### The result objects
The distance result contains the pair of witness points $p_1$ and $p_2$, the normal direction pointing from $p_1$ to $p_2$ and the signed distance.

In [17]:
d = geom_data.distanceResults[0]
print("p1:",d.getNearestPoint1())
print("p2:",d.getNearestPoint2())
print("n:",d.normal)
print("dist:",d.min_distance)
print("check:",np.cross(d.normal,d.getNearestPoint2()-d.getNearestPoint1())) # The two vectors are parallel.

The collision object contains similar information, but can stores several witness pairs instead of a single one, or none if there is no collisions.
The collision is decided based on a security margin, tuned in the collision request object.

In [18]:
print('Margin:',geom_data.collisionRequests[0].security_margin)
c = geom_data.collisionResults[0]
print('Number of collisions:',len(c.getContacts()))

Now choose a configuration in collision and look at the content of this contact list.

In [19]:
# %load tp4/generated/example_find_collisions_find
for trial in range(1000):
    q = pin.randomConfiguration(model)
    col = pin.computeCollisions(model,data,geom_model,geom_data,q)
    if col: break
assert(col)
viz.display(q)
